In [ ]:
pip install opencv-python numpy pandas easyocr pillow

In [25]:
import json
import os
import pandas as pd

with open('video_info.json', "r", encoding="utf-8") as file:
        data = json.load(file)

In [29]:
flattened_data = [video for sublist in data for video in sublist]
df = pd.DataFrame(flattened_data)
df

,title,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,category_id,duration
0,80세할머니 직장에서 퇴근시켜드리기,2y2A96ZQGXU,2024-07-12,https://i.ytimg.com/vi/2y2A96ZQGXU/hqdefault.jpg,32175761,1353908,8854,24,PT1M
1,어르신들께 삼계탕100마리 모셨습니다,In6tXXgu-gE,2024-07-15,https://i.ytimg.com/vi/In6tXXgu-gE/hqdefault.jpg,14988361,486791,7079,24,PT57S
2,80세 달고나할머니 다시찾아가봤습니다,b14f7A-mn0c,2024-10-25,https://i.ytimg.com/vi/b14f7A-mn0c/hqdefault.jpg,17811708,945586,8491,24,PT59S
3,8남매에게 집을 사주는 보겸,UUdBdQkH-ns,2024-07-08,https://i.ytimg.com/vi/UUdBdQkH-ns/hqdefault.jpg,11096380,465775,7199,24,PT1M
4,8남매에게 집을 새집으로 바꿔줬습니다,1zpA-hwn18I,2024-07-07,https://i.ytimg.com/vi/1zpA-hwn18I/hqdefault.jpg,10820141,237481,15668,24,PT8M53S
...,...,...,...,...,...,...,...,...,...
160,(4K) 제니(JENNIE) &#39;like JENNIE&#39; | NPOP L...,lBimDKKNDDs,2025-03-09,https://i.ytimg.com/vi/lBimDKKNDDs/hqdefault.jpg,2767899,225233,5332,22,PT2M17S
161,[TEASER] 제니(JENNIE) &#39;like JENNIE&#39; 최초공개...,85DSFgZ4-wQ,2025-03-08,https://i.ytimg.com/vi/85DSFgZ4-wQ/hqdefault.jpg,1036625,68967,1296,22,PT35S
162,(4K) MEOVV(미야오) &#39;MEOW&#39; l NPOP LIMITED ...,3yGa0jhUJRU,2024-09-08,https://i.ytimg.com/vi/3yGa0jhUJRU/hqdefault.jpg,879992,30287,890,22,PT3M26S
163,키스오브라이프 핫걸들의 유치뽕짝 TMI 폭로 파티🔥 l [시끌벅적 뒤에서 말해요] ...,zgkBhEUig90,2024-07-08,https://i.ytimg.com/vi/zgkBhEUig90/hqdefault.jpg,342721,10011,459,22,PT18M31S


In [33]:
import isodate

def convert_duration(iso_duration):
    # ISO 8601 형식 변환
    duration = isodate.parse_duration(iso_duration)
    
    # 시, 분, 초로 변환
    total_seconds = int(duration.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # HH:MM:SS 형식으로 변환
    return f"{hours:02}:{minutes:02}:{seconds:02}"

dur=[]
for i in df['duration']:
    dur.append(convert_duration(i))
df['duration']=dur
df

,title,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,category_id,duration
0,80세할머니 직장에서 퇴근시켜드리기,2y2A96ZQGXU,2024-07-12,https://i.ytimg.com/vi/2y2A96ZQGXU/hqdefault.jpg,32175761,1353908,8854,24,00:01:00
1,어르신들께 삼계탕100마리 모셨습니다,In6tXXgu-gE,2024-07-15,https://i.ytimg.com/vi/In6tXXgu-gE/hqdefault.jpg,14988361,486791,7079,24,00:00:57
2,80세 달고나할머니 다시찾아가봤습니다,b14f7A-mn0c,2024-10-25,https://i.ytimg.com/vi/b14f7A-mn0c/hqdefault.jpg,17811708,945586,8491,24,00:00:59
3,8남매에게 집을 사주는 보겸,UUdBdQkH-ns,2024-07-08,https://i.ytimg.com/vi/UUdBdQkH-ns/hqdefault.jpg,11096380,465775,7199,24,00:01:00
4,8남매에게 집을 새집으로 바꿔줬습니다,1zpA-hwn18I,2024-07-07,https://i.ytimg.com/vi/1zpA-hwn18I/hqdefault.jpg,10820141,237481,15668,24,00:08:53
...,...,...,...,...,...,...,...,...,...
160,(4K) 제니(JENNIE) &#39;like JENNIE&#39; | NPOP L...,lBimDKKNDDs,2025-03-09,https://i.ytimg.com/vi/lBimDKKNDDs/hqdefault.jpg,2767899,225233,5332,22,00:02:17
161,[TEASER] 제니(JENNIE) &#39;like JENNIE&#39; 최초공개...,85DSFgZ4-wQ,2025-03-08,https://i.ytimg.com/vi/85DSFgZ4-wQ/hqdefault.jpg,1036625,68967,1296,22,00:00:35
162,(4K) MEOVV(미야오) &#39;MEOW&#39; l NPOP LIMITED ...,3yGa0jhUJRU,2024-09-08,https://i.ytimg.com/vi/3yGa0jhUJRU/hqdefault.jpg,879992,30287,890,22,00:03:26
163,키스오브라이프 핫걸들의 유치뽕짝 TMI 폭로 파티🔥 l [시끌벅적 뒤에서 말해요] ...,zgkBhEUig90,2024-07-08,https://i.ytimg.com/vi/zgkBhEUig90/hqdefault.jpg,342721,10011,459,22,00:18:31


In [55]:
import cv2
import numpy as np
import easyocr
import requests
from io import BytesIO
from PIL import Image
from sklearn.cluster import KMeans

# OCR 모델 초기화 (한국어 & 영어 지원)
reader = easyocr.Reader(['ko', 'en'])

# 얼굴 인식 모델 초기화 (OpenCV에서 제공하는 Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [56]:
def extract_image_info(image_url):
    """썸네일 이미지에서 텍스트, 얼굴, 색상 정보 추출"""
    try:
        # 🔹 1. 이미지 로드
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        image_cv = np.array(image.convert("RGB"))  # PIL → OpenCV 변환

        # 🔹 2. 텍스트 검출 (OCR)
        text_results = reader.readtext(image_cv)

        # 🔹 3. 얼굴 검출
        gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # 텍스트 및 얼굴 정보 초기화
        text_details = []
        largest_text = ""
        max_area = 0
        contains_face = len(faces) > 0
        num_face = len(faces)

        for result in text_results:
            bbox, text, _ = result  # bbox: [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
            x1, y1 = bbox[0]
            x3, y3 = bbox[2]

            width = abs(x3 - x1)
            height = abs(y3 - y1)
            area = width * height  # 텍스트 크기 (바운딩 박스 면적)

            # 🔹 4. 텍스트 색상 추출 (ROI 평균 색상 계산)
            roi = image_cv[int(y1):int(y3), int(x1):int(x3)]
            if roi.size != 0:
                avg_color = roi.mean(axis=(0, 1))  # (B, G, R) 평균
                avg_color = tuple(map(int, avg_color))  # 정수 변환
            else:
                avg_color = (0, 0, 0)  # ROI가 비어있으면 검정색 처리

            # 🔹 5. 가장 큰 텍스트 업데이트
            if area > max_area:
                max_area = area
                largest_text = text

            # 🔹 6. 모든 텍스트 정보 저장
            text_details.append({
                "text": text,
                "width": width,
                "height": height,
                "area": area,
                "avg_color": avg_color  # 평균 색상 저장
            })

        # 🔹 7. 주 색상 3개 추출
        # 이미지 리사이징 및 색상 분석을 위한 처리
        image_resized = cv2.resize(image_cv, (200, 200))  # 크기 줄여서 성능 개선
        image_reshaped = image_resized.reshape((-1, 3))  # (W * H, 3)
        
        # KMeans로 주요 색상 추출
        kmeans = KMeans(n_clusters=3)
        kmeans.fit(image_reshaped)
        dominant_colors = kmeans.cluster_centers_.astype(int)  # 중심값 (주 색상 3개)
        
        # 주 색상은 BGR로 추출되므로 RGB로 변환하여 반환
        dominant_colors_rgb = [tuple(color[::-1]) for color in dominant_colors]

        return {
            "contains_text": bool(text_results),  # 텍스트 존재 여부
            "contains_face": contains_face,  # 얼굴 존재 여부
            "num_face": num_face,  # 얼굴 수
            "text_details": text_details,  # 텍스트 관련 세부 정보
            "largest_text": largest_text,  # 가장 큰 텍스트
            "dominant_colors": dominant_colors_rgb  # 주 색상 3개 (RGB)
        }

    except Exception as e:
        print(f"이미지 분석 실패: {e}")
        return {
            "contains_text": False,
            "contains_face": False,
            "num_face": 0,
            "text_details": [],
            "largest_text": "",
            "dominant_colors": []
        }

In [ ]:
# 썸네일 분석 실행
df["analysis"] = df["thumbnail_url"].apply(extract_image_info)

# 개별 컬럼 분리
df["contains_text"] = df["analysis"].apply(lambda x: x["contains_text"])
df["contains_face"] = df["analysis"].apply(lambda x: x["contains_face"])
df["num_face"] = df["analysis"].apply(lambda x: x["num_face"])
df["text_details"] = df["analysis"].apply(lambda x: x["text_details"])
df["largest_text"] = df["analysis"].apply(lambda x: x["largest_text"])
df["dominant_colors"] = df["analysis"].apply(lambda x: x["dominant_colors"])

# 필요 없는 컬럼 삭제
df.drop(columns=["analysis"], inplace=True)

In [87]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder

# 이미지 명도 (Brightness)
def calculate_brightness(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray_image)  # Average brightness
    return brightness

# 이미지 대비 (Contrast)
def calculate_contrast(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = np.std(gray_image)  # Standard deviation as contrast
    return contrast

# 주요 객체 추출 (Main Object Detection)
def detect_main_objects(image):
    model = keras.applications.MobileNetV2(weights='imagenet', include_top=True)
    image_resized = cv2.resize(image, (224, 224))
    image_array = img_to_array(image_resized)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = keras.applications.mobilenet_v2.preprocess_input(image_array)
    
    predictions = model.predict(image_array)
    decoded_predictions = keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]
    main_object = decoded_predictions[0][1]  # 가장 확률이 높은 객체
    return main_object

# 중앙 집중 분석 (Central Focus)
def check_central_focus(image, object_bbox):
    h, w, _ = image.shape
    object_center = [(object_bbox[0] + object_bbox[2]) / 2, (object_bbox[1] + object_bbox[3]) / 2]
    image_center = [w / 2, h / 2]
    
    distance = np.linalg.norm(np.array(object_center) - np.array(image_center))
    return distance

# 텍스트와 객체의 비율 (Text-Object Ratio)
def text_object_ratio(image, text_bbox, object_bbox):
    text_area = (text_bbox[2] - text_bbox[0]) * (text_bbox[3] - text_bbox[1])
    object_area = (object_bbox[2] - object_bbox[0]) * (object_bbox[3] - object_bbox[1])
    return text_area / object_area

# 사람의 표정 및 행동 분석 (Emotion and Action of Person)
def analyze_person_expression_and_action(image):
    # 얼굴 인식 (여기서는 OpenCV의 haarcascades로 얼굴 감지)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    emotions = []  # 감정 분석 결과를 저장할 리스트 (예: 'happy', 'sad', 'surprised')
    actions = []  # 특정 행동 (예: 'waving', 'pointing')
    
    # 감정 분석, 동작 분석은 얼굴 인식된 부분에 대해 더 고급 모델을 사용할 수 있음
    # 예시로 얼굴을 인식하고 감정이 표출된 부분만 나타냄
    if len(faces) > 0:
        # 예시: 얼굴 감지된 경우 긍정적인 표정일 가능성 확인
        emotions.append('happy')  # 이건 실제 감정 분석 모델을 적용해야 함
    else:
        emotions.append('no_face')
    
    return emotions, actions

# 이미지 파일을 불러와서 분석하는 메인 함수
def analyze_thumbnail(image_path):
    image = cv2.imread(image_path)
    
    # 1. 이미지 명도
    brightness = calculate_brightness(image)
    
    # 2. 이미지 대비
    contrast = calculate_contrast(image)
    
    # 3. 주요 객체 추출
    main_object = detect_main_objects(image)
    
    # 4. 객체의 중앙 집중도 분석
    object_bbox = [50, 50, 150, 150]  # 예시로 지정된 객체의 bounding box (x1, y1, x2, y2)
    central_focus = check_central_focus(image, object_bbox)
    
    # 5. 텍스트와 객체의 비율
    text_bbox = [10, 10, 200, 50]  # 예시 텍스트 bounding box (x1, y1, x2, y2)
    ratio = text_object_ratio(image, text_bbox, object_bbox)
    
    # 6. 사람의 표정 및 행동 분석
    emotions, actions = analyze_person_expression_and_action(image)
    
    # 결과 출력
    return {
        'brightness': brightness,
        'contrast': contrast,
        'main_object': main_object,
        'central_focus_distance': central_focus,
        'text_object_ratio': ratio,
        'emotions': emotions,
        'actions': actions
    }

# 사용 예시
image_path = 'https://i.ytimg.com/vi/umSVYiHcJcQ/hqdefault.jpg'  # 썸네일 이미지 경로
def get_normal_image(self,url):
	image_nparray = np.asarray(bytearray(requests.get(url).content), dtype=np.uint8)
        image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
        return image
image=get_normal_image(image_path)

analysis_result = analyze_thumbnail(image_path)

# 분석 결과 출력
print(analysis_result)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [96]:
image_path = 'https://i.ytimg.com/vi/umSVYiHcJcQ/hqdefault.jpg'  # 썸네일 이미지 경로

def get_normal_image(self,url):
    image_nparray = np.asarray(bytearray(requests.get(url).content), dtype=np.uint8)
    image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
    return image

image=get_normal_image(image_path)

NameError: name 'self' is not defined